In [2]:
from pprint import pprint
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import itertools
import seaborn as sns

# Import the data
nyc_registry = pd.read_csv('../dogdata/NYC_Dog_Licensing_Dataset_2016-edit.csv')
coren = pd.read_csv('../dogdata/coren-edit.csv')
nyc_census = pd.read_csv('../censusdata/ACS_16_1YR_S0201_with_ann-edit.csv') # use 2016 data
edmonton_registry = pd.read_csv('../dogdata/Edmonton_Pet_Licenses_by_Neighbourhood_2018-edit.csv')
adelaide_registry = pd.read_csv('../dogdata/Dog_Registrations_Adelaide_2016-edit.csv')
wiki = pd.read_csv('../dogdata/wiki-edit.csv')
turcsan = pd.read_csv('../dogdata/turcsan.csv')

In [14]:
turcsan.corr()

,Calm,Trainable,Sociable,Bold
Calm,1.000000,-0.596697,0.864370,0.753806
Trainable,-0.596697,1.000000,-0.331093,-0.409075
Sociable,0.864370,-0.331093,1.000000,0.815936
Bold,0.753806,-0.409075,0.815936,1.000000


In [5]:
# Strip out dirty values
nyc_registry['Borough'] = nyc_registry['Borough'].map(lambda x: None if x not in {'Brooklyn', 'Bronx', 'Staten Island', 'Manhattan', 'Queens'} else x)
nyc_registry['BreedName'] = nyc_registry['BreedName'].map(lambda x: None if x == 'Unknown' else x)
nyc_registry.dropna(inplace=True)

In [6]:
attrib = coren.set_index('Breed').join(turcsan.set_index('Breed'), how='outer')
nyc_attrib = nyc_registry.set_index('BreedName').join(attrib, how='left')
nyc_attrib = nyc_attrib[['Borough','Calm', 'Trainable', 'Sociable', 'Bold', 'Obedient']]
nyc_attrib_g = nyc_attrib.groupby('Borough')
nyc_attrib_g_mean = nyc_attrib_g.mean()
nyc_attrib_g_mean

,Calm,Trainable,Sociable,Bold,Obedient
Borough,,,,,
Bronx,0.378205,0.370192,0.381410,0.586538,0.505029
Brooklyn,0.458333,0.419643,0.462054,0.604167,0.484541
Manhattan,0.490585,0.467790,0.506442,0.617939,0.524446
Queens,0.401709,0.402564,0.415385,0.576923,0.513731
Staten Island,0.565359,0.436275,0.568627,0.681373,0.541384


In [13]:
nyc_attrib_g_mean.iloc[0].values

array([0.37820513, 0.37019231, 0.38141026, 0.58653846, 0.50502874])

In [14]:
f, p = stats.f_oneway(nyc_attrib_g_mean.iloc[0].values,
                      nyc_attrib_g_mean.iloc[1].values,
                      nyc_attrib_g_mean.iloc[2].values,
                      nyc_attrib_g_mean.iloc[3].values,
                      nyc_attrib_g_mean.iloc[4].values)
 
print ('One-way ANOVA')
print ('=============')
print ('F value:', f)
print ('P value:', p, '\n')

One-way ANOVA
F value: 1.6768563660187459
P value: 0.1947447193366099 



## P-value > 0.05, so I fail to reject the null hypothesis